## 한글 텍스트 처리(konlpy)
- 이것 실행을 위해서는 3가지 설치 필요(책 참고) : 1.java-jpype1 설치-pip install / 2. jdk를 다운로드 / 3. jdk의 jre파일을 환경변수 설정(JAVA_HOME)
- 마지막으로 아나콘다 프롬프트에 'pip install konlpy'를 치고 오류가 나지 않으면 정상적으로 설치된 것.
- 앞의 텍스트 처리는 전부 영어였는데, 한글로 텍스트 처리를 할 수 있는 유일한 패키지임

In [1]:
import konlpy

- 순서: 데이터 불러오기 및 전처리 -> 피처벡터화 -> ML 모델(여기서는 지도학습-분류 = 지도학습 기반 감성분석)

In [2]:
#데이터 불러오기('ratings'파일) - train셋(X: documnet, y: label)
import pandas as pd
train_df = pd.read_csv('08/ratings/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
#label 비율
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [4]:
train_df.isnull().sum() #documnet의 null값 처리-> 공백으로
                                                     #숫자값 처리-> 공백으로

id          0
document    5
label       0
dtype: int64

- (파이썬 정규표현식 참고) http://recordingbetter.com/python/2017/05/23/Python-%EC%A0%95%EA%B7%9C%ED%91%9C%ED%98%84%EC%8B%9D

In [ ]:
import re

train_df['document'].fillna(' ', inplace=True) #null값 처리
train_df['documnet'] = train_df['document'].apply(lambda x : re.sub(r"\d+", " ", x)) #숫자값 처리
train_df.isnull().sum()

id          0
document    0
label       0
documnet    0
dtype: int64

In [ ]:
#test셋
test_df = pd.read_csv('08/ratings/ratings_test.txt', sep='\t')
display(test_df.head(3))
print(test_df.isnull().sum())

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


id          0
document    3
label       0
dtype: int64


In [ ]:
test_df['document'].fillna(' ', inplace=True) #null값 처리
test_df['documnet'] = test_df['document'].apply(lambda x : re.sub(r"\d+", " ", x)) #숫자값 처리
test_df.isnull().sum()

id          0
document    0
label       0
documnet    0
dtype: int64

In [ ]:
X_train = train_df['document']
y_train = train_df['label']
X_test = test_df['document']
y_test = test_df['label']

- 전처리: tokenize ==> Twitter객체의 morphs() 메서드를 사용 시 형태소 단어 형태로 토큰화해 list로 반환

In [ ]:
from konlpy.tag import Twitter

#객체
twitter = Twitter()
#token화
def tw_tokenizer(text):
    return twitter.morphs(text) #객체.morphs()는 형태소 형태로 토큰화

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


- 피처벡터화: tf-idf로 하되, tokenizer 함수를 위의 tw_tokenizer 사용

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
#tf-idf수행-- 항상 tfidf는 느림 주의......
#객체
tfidf = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
#학습
tfidf.fit(X_train)
#적용(2번)
X_train_vect = tfidf.transform(X_train)
X_test_vect = tfidf.transform(X_test)

- ML 적용: 분류-로지스틱

In [ ]:
#분류모델은 로지스틱 - 하이퍼 파라미터 튜닝은 gridsearch
#객체
lg_clf = LogisticRegression(random_state=0)
#파라미터- C
params = {'C': [1, 3.5, 4.5, 5.5, 10]}
#그리드서치 객체
grid_cv = GridSearchCV(lg_clf, params, cv=3, scoring='accuracy', verbose=1)
#학습
grid_cv.fit(X_train_vect, y_train)
#cv 결과
print(grid_cv.best_params_, grid_cv.best_score_) #C: 3.5인 게 가장 최적의 파라미터

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.0min finished


{'C': 3.5} 0.8628733333333334


In [ ]:
#위의 하이퍼파라미터 튜닝 결과를 이용해 최종 감성 분석 예측!
#다시 예측
pred = grid_cv.predict(X_test_vect)
#평가
np.round(accuracy_score(pred, y_test), 4)

0.8654

- <번외> pipeline으로 만들어보기 + gridsearch // tfidf가 느릴땐 두개 연합해서 하는것도 방법!

In [ ]:
#gridsearch없는버젼 - 하이퍼파라미터 설정함.
from sklearn.pipeline import Pipeline
#객체
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)),
             ('logistic', LogisticRegression(C=3.5, random_state=0))])
#학습-학습예측은 원본데이터를 넣자!
pipeline.fit(X_train, y_train)
#예측
preds = pipeline.predict(X_test)
#평가
np.round(accuracy_score(preds, y_test), 4)

0.8653

In [ ]:
#gridsearch버젼
from sklearn.pipeline import Pipeline
#객체
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)),
             ('logistic', LogisticRegression(random_state=0))])
#파라미터
params = {'logistic__C': [1, 3.5, 4.5, 5.5, 10]}
#그리드서치 객체
grid_pipe = GridSearchCV(pipeline, params, cv=3, scoring='accuracy', verbose=1)
#학습-학습예측은 원본데이터를 넣자!
grid_pipe.fit(X_train, y_train)
#cv 결과
print(grid_pipe.best_estimator_)
#예측
preds = grid_pipe.predict(X_test)
#평가
print(np.round(accuracy_score(preds, y_test), 4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
